# Projet maison n° 3

In [1]:
# imports
import pandas as pd
import numpy as np
from pandas import DataFrame
from pathlib import Path
import sys
import re
import os
from os.path import basename

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [2]:
dossier_source = Path("names\\")
filesUS = dossier_source.glob("*.txt")

In [3]:
#Fonction qui extrait l'année correspondant à chaque fichier
def annee(fichier):
    p = re.compile('.*yob(....).txt')
    return p.match(os.path.basename(fichier)).group(1)

In [4]:
# names us
def df_names_us():
    df = pd.DataFrame(columns=['year', 'name', 'gender', 'births'])
    for file in filesUS:
        df1 = pd.read_csv(file, names=['name', 'gender', 'births'])
        yob = annee(file)
        df1.insert(0,'year',yob)
        df = df.append(df1, ignore_index=True)
    return df

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [5]:
#On définit le path jusqu'au fichier
fileFR = Path("nat2020.csv")

In [6]:
# names fr
def df_names_fr():
    df_FR = pd.read_csv(fileFR, sep=';')
    
    #Nom et ordre des colonnes
    df_FR = df_FR.rename(columns={"sexe": "gender", "preusuel": "name",  "annais": "year",  "nombre": "births"})
    df_FR = df_FR[['year','name','gender','births']]
    
    df_FR['gender']=df_FR['gender'].map({1:"M", 2:"F"})
    
    #même type que les données US
    df_FR.astype('object').dtypes
    
    #Suppression des prénoms à une lettre, es "prénoms rares", des années "XXXX"
    df_FR = df_FR.drop(df_FR[df_FR['name'].str.len() == 1].index)
    df_FR = df_FR.drop(df_FR[df_FR['name'] == "_PRENOMS_RARES"].index)    
    df_FR = df_FR.drop(df_FR[df_FR['year'] == "XXXX"].index)    

    #Changement de la casse des prénoms
    df_FR["name"] = df_FR["name"].apply(lambda x: str(x).title())
    
    #Tri
    df_FR = df_FR.sort_values(by=['year','gender','births','name'], ascending=[True, True, False, True])
    
    #index
    df_FR.index = pd.RangeIndex(stop=len(df_FR.index))
 
    return df_FR

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [7]:
# Fonction pour traduire les strings en ISO quand c'est possible
def iso(string):
    regexISO = re.compile('^.*\(([A-Z]{3})\)')
    if regexISO.match(string):
        return(regexISO.match(string).group(1))
    else: return(string)

In [8]:
# taux de change
def df_taux_change(devises):
    #on importe le fichier source
    data = pd.read_csv("Webstat_Export_20210929.csv", sep=";")
    #on change les noms des colonnes pour correspondre à un code ISO, et on enlève les premières lignes
    data = data.rename(mapper=iso,axis="columns")
    data = data.drop(list(range(5)))
    
    #Transformation des dates en "datetime" et utilisation comme index
    data["Titre :"] = pd.to_datetime(data["Titre :"], infer_datetime_format=True)
    data.set_index("Titre :", inplace=True)
    
    #Conversion de tous les taux en floats
    data.replace([",", "-"], [".", np.nan], inplace=True, regex=True)
    data = data.astype('float')
    
    #suppression des lignes contenant des NaN
    data = data.dropna()
    
    df = data[devises]
    return df

### Tests

In [9]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [10]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [11]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 16.148s

OK
